# Source and channel coding


Ne fonctionne pas encore. Problème avec le X2 et le X4 (type du X4).

In [19]:
% Programme II
%               ***************
%               *             *
% Message X --> * Source Code * --> X2
%               *             *
%               ***************
% X2 is supposed known

% Warning : The output of LempelZivEnco is a 'string'

% Input : X2
X2 = "01101111010001";

clear X3 X4 Y1 Y2;

for i = 1:length(X2) - 1
    X4(i) = str2num(X2(i))
end

% X4 is a binary sequence "=" X2
X4
% X4(1:10)
% X2(1:10)


Unrecognized function or variable 'X4'.



In [24]:
%               ****************
%               *              *
% Message X --> * Channel Code * --> X5
%               *              *
%               ****************

% Let G be a generator matrix of a block code
G = [1 0 1 0 1 0; 0 1 0 1 0 1]; % repetition
u = [0 1; 1 1; 1 0];
distmin = min(sum(rem(u * G, 2), 2))

% Code X4
% Checking the length of the sequence
L4 = length(X4);
rest = rem(L4, size(G, 1));

if rest
    X4(L4 + 1:L4 + size(G, 1) - rest) = zeros(1, size(G, 1) - rest);
end

size(X4)

% Verification is possible by vec2mat(mat,padded) of Matlab
mat = vec2mat(X4, size(G, 1));
size(mat)
X4(1:10)
mat(1:5, :)

code = rem(mat * G, 2);
X5 = reshape(code', 1, size(code, 1) * size(code, 2));
Rcb = length(X5) / length(X4) % Code rate


Unrecognized function or variable 'X4'.



In [25]:
%                       ***********
%                       *         *
% Message envoyé X5 --> * Channel * --> Y1
%                       *         *
%                       ***********

% Canal sans bruit ==> Y1 = X5
Y1 = X5;

%                     ******************
%                     *                *
% Message reçu Y1 --> * Decode Channel * --> Y2
%                     *                *
%                     ******************

% Decode Y1
Gri = G' * inv(G * G') % right inverse
G * Gri
Ym = vec2mat(Y1, size(G, 2));
deco = Ym * Gri;

Y2 = reshape(deco', 1, size(deco, 1) * size(deco, 2)); % real vector
Y2 = (Y2 > 0.5); % binary vector after a threshold

BER = sum((Y2 - X4) .^ 2) % Bit Error Rate

%                     ************
%                     *          *
% Sent Message X5 --> * Canal II * --> Y1
%                     *          *
%                     ************

% Noisy channel ==> Y1 = X5 + noise
e = zeros(size(X5));

for i = 1:length(X5)
    e(i) = (mod(i, size(G, 2)) == 1);
end

Nbrerr = sum(e)
Y1 = rem(X5 + e, 2);

%                         ****************
%                         *              *
% Received Message Y1 --> * Decode Canal * --> Y2
%                         *              *
%                         ****************

% Correction Y1
% Control matrix H
% G is in systematic form
P = zeros(size(G, 1), size(G, 2) - size(G, 1));

for i = 1:size(G, 1)
    P(i, :) = G(i, size(G, 1) + 1:end);
end

% H=[P^T I] is a n-k x k matrix
Pt = P';
I = eye(size(G, 2) - size(G, 1));
H = zeros(size(G, 2) - size(G, 1), size(G, 1));

for i = 1:size(G, 2) - size(G, 1)
    H(i, 1:size(Pt, 2)) = Pt(i, 1:end);
    H(i, size(Pt, 2) + 1:end) = I(i, 1:end);
end

rem(G * H', 2) % G x H^T = 0

% error check
mat=vec2mat(Y1,size(G,2));
s=rem(mat*H',2); % Syndrome
inder=(sum(s,2)>0); % error indicator

% error correction
matc=mat; %mat corrected
for i =1:size(mat,1)
    if inder(i) % correction
        a=0;
        b=0;
        for j=1:3 %repetition code 2x6
            a=a+mat(i,2*j);
            b=b+mat(i,2*j-1);
        end
        for j=1:3
            matc(i,2*j)=((a/3)>0.5);
            matc(i,2*j-1)=((b/3)>0.5);
        end
    end
end

% Y1 corrected
Y1=reshape(matc',1,size(matc,1)*size(matc,2));
BER=sum((Y1-X5).^2) % Bit Error Rate

% Decode Y1
deco=matc*Gri;

Y2=reshape(deco',1,size(deco,1)*size(deco,2)); % real vector
Y2=(Y2>0.5); % binary vector after a threshold

BER=sum((Y2-X4).^2) % Bit Error Rate

%                              *****************
%                              *               *
% Channel Decoding Y2 = X2 --> * Decode Source * --> Y3 = X
%                              *               *
%                              *****************

% format string
str=num2str(Y2);
for i=1:length(Y2)
    Y2s(i)=str(3*(i-1)+1);
end

Y3=LempelZivDeco(Y2s,Dict2); % Y2s doit être un string

% Attention : La sortie de LempelZivDeco est un 'string'
for i =1:length(Y3)
    Y4(i)=str2num(Y3(i));
end

BERF=sum((X(1:length(Y3))-Y4).^2) % Bit Error Rate

Unrecognized function or variable 'X5'.

